# Notebook 4 - pandas
[pandas](http://pandas.pydata.org) provides high-level data structures and functions designed to make working with structured or tabular data fast, easy and expressive. The primary objects in pandas that we will be using are the `DataFrame`, a tabular, column-oriented data structure with both row and column labels, and the `Series`, a one-dimensional labeled array object.

pandas blends the high-performance, array-computing ideas of NumPy with the flexible data manipulation capabilities of spreadsheets and relational databases. It provides sophisticated indexing functionality to make it easy to reshape, slice and perform aggregations.

While pandas adopts many coding idioms from NumPy, the most significant difference is that pandas is designed for working with tabular or heterogeneous data. NumPy, by contrast, is best suited for working with homogeneous numerical array data.
<br>

## Table of Contents:
- [Data Structures](#structures)
    - [Series](#series)
    - [DataFrame](#dataframe)
- [Essential Functionality](#ess_func)
    - [Removing Entries](#removing)
    - [Indexing](#indexing)
    - [Arithmetic Operations](#arithmetic)
- [Summarizing and Computing Descriptive Statistics](#sums)
- [Loading and storing data](#loading)
- [Data Cleaning and preperation](#cleaning)
    - [Handling missing data](#missing)
    - [Data transformation](#transformation)

The common pandas import statment is shown below:

In [1]:
# Common pandas import statement
import numpy as np
import pandas as pd

# Data Structures <a name="structures"></a>
## Series <a name="series"></a>
A Series is a one-dimensional array-like object containing a sequence of values and an associated array of data labels called its index.

The easiest way to make a Series is from an array of data:

In [2]:
data = pd.Series([4, 7, -5, 3])

Now try printing out data

In [3]:
data

0    4
1    7
2   -5
3    3
dtype: int64

The string representation of a Series displayed interactively shows the index on the left and the values on the right. Because we didn't specify an index, the default one is simply integers 0 through N-1.

It is possible to get only the index or only the values of a Serioes with:

In [4]:
data.values

array([ 4,  7, -5,  3])

You can specify custom indices when intialising the Series

In [5]:
data2 = pd.Series([4, 7, -5, 3], index=["a", "b", "c", "d"])

In [6]:
data2

a    4
b    7
c   -5
d    3
dtype: int64

Another way to think about Series is as a fixed-length ordered dictionary. Furthermore, you can actually define a Series in a similar manner to a dictionary

In [7]:
cities = {"Glasgow" : 599650, "Edinburgh" : 464990, "Aberdeen" : 196670, "Dundee" : 147710}
data3 = pd.Series(cities)

In [8]:
data3

Glasgow      599650
Edinburgh    464990
Aberdeen     196670
Dundee       147710
dtype: int64

## DataFrame <a name="dataframe"></a>
A DataFrame represents a rectangular table of data and contains an ordered collection of columns, each of which can be a different value type. The DataFrame has both row and column index and can be thought of as a dict of Series all sharing the same index.

The most common way to create a DataFrame is with dicts:

In [9]:
data = {"cities" : ["Glasgow", "Edinburgh", "Aberdeen", "Dundee"],
        "population" : [599650, 464990, 196670, 147710],
        "year" : [2011, 2013, 2013, 2013]}
frame = pd.DataFrame(data)

Try printing it out

In [10]:
frame

,cities,population,year
0,Glasgow,599650,2011
1,Edinburgh,464990,2013
2,Aberdeen,196670,2013
3,Dundee,147710,2013


Jupyter Notebooks prints it out in a nice table but the basic version of this is also just as readable!

Additionally you can also specify the order of columns, or the row index,  during initialisation

In [11]:
frame2 = pd.DataFrame(data, columns=["year", "cities", "population"], index=["a", "b", "c", "d"])
frame2

,year,cities,population
a,2011,Glasgow,599650
b,2013,Edinburgh,464990
c,2013,Aberdeen,196670
d,2013,Dundee,147710


You can retrieve a particular column from a DataFrame with
```python
frame["cities"]
```
The result is going to be a Series.

In [12]:
frame2["cities"]

a      Glasgow
b    Edinburgh
c     Aberdeen
d       Dundee
Name: cities, dtype: object

It is also possible to add and modify the columns of a DataFrame

In [13]:
frame2["size"] = 100
frame2

,year,cities,population,size
a,2011,Glasgow,599650,100
b,2013,Edinburgh,464990,100
c,2013,Aberdeen,196670,100
d,2013,Dundee,147710,100


In [14]:
frame2["size"] = [175, 264, 65.1, 60]  # in km^2
frame2

,year,cities,population,size
a,2011,Glasgow,599650,175.0
b,2013,Edinburgh,464990,264.0
c,2013,Aberdeen,196670,65.1
d,2013,Dundee,147710,60.0


Here is a table of different ways of initialising a DataFrame for your reference

| Type | Notes |
| --- | --- |
| 2D ndarray | A matrix of data; passing optional row and column labels |
| dict of arrays, lists, or tuples | Each sequence becomes a column in the DataFrame; all sequences must be the same length |
| dict of Series | Each value becomes a column; indexes from each Series are unioned together to<br>form the result's row index if not explicit index is passed |
| dict of dicts | Each inner dict becomes a column; keys are unioned to form the row<br>index as in the "dict of Series" case |
| List of dicts or Series | Each item becomes a row in the DataFrame; union of dict keys or<br>Series indices becomes the DataFrame's column labels |
| List of lists or tuples | Treated as the "2D ndarray" case |

# Essential Functionality <a name="ess_func"></a>
In this section, we will go through the fundamental mechanics of interacting with the data contained in a Series or DaraFrame.

## Removing columns/indices <a name="removing"></a>
Similar to above, it is easy to remove entries. This is done with the `drop()` method and can be applied to both columns and indices:

In [15]:
# define new DataFrame
data = np.reshape(np.arange(9), (3,3))
df = pd.DataFrame(data, index=["a", "b", "c"],
                  columns=["Edinburgh", "Glasgow", "Aberdeen"])

df

,Edinburgh,Glasgow,Aberdeen
a,0,1,2
b,3,4,5
c,6,7,8


In [16]:
df.drop("b")  # remove row (index)

,Edinburgh,Glasgow,Aberdeen
a,0,1,2
c,6,7,8


In [17]:
# You can also drop from a column
df.drop(["Aberdeen", "Edinburgh"], axis="columns")

,Glasgow
a,1
b,4
c,7


Note that the original data frame is unchanged: `df.drop()` gives us a new data frame with the desired data dropped, and leaves the original data intact. We can ask `.drop()` to operate directly on the original data frame by setting the argument `inplace=True`.

In [18]:
df

,Edinburgh,Glasgow,Aberdeen
a,0,1,2
b,3,4,5
c,6,7,8


## Indexing <a name="indexing"></a>

Indexing in pandas works simiarly to numpy, but we have to use `.iloc` instead of just normal indexing

In [19]:
data = np.reshape(np.arange(9), (3,3))
df = pd.DataFrame(data, index=["a", "b", "c"],
                  columns=["Edinburgh", "Glasgow", "Aberdeen"])

df

,Edinburgh,Glasgow,Aberdeen
a,0,1,2
b,3,4,5
c,6,7,8


In [20]:
df[2]   # this won't work

KeyError: 2

In [21]:
df.iloc[2]  # this works and return a series

Edinburgh    6
Glasgow      7
Aberdeen     8
Name: c, dtype: int64

You can also slice a dataframe as usual:

In [22]:
df.iloc[:2]

,Edinburgh,Glasgow,Aberdeen
a,0,1,2
b,3,4,5


You can index into many dimensions as seen in NumPy:

In [23]:
df.iloc[2,0]

6

Finally, when the indices/column names of your data frame are not integers, you can use `.loc` to index:

In [24]:
frame2

,year,cities,population,size
a,2011,Glasgow,599650,175.0
b,2013,Edinburgh,464990,264.0
c,2013,Aberdeen,196670,65.1
d,2013,Dundee,147710,60.0


In [25]:
frame2.loc["a", "size"]

175.0

### Exercise 1
A dataset of random numbers is created below. Obtain all rows starting from row 85 to 97.

*Note: Remember that Python uses 0-based indexing*

In [26]:
df = pd.DataFrame(np.reshape(np.arange(10000), (100,100)))
df.iloc[85:98]

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
85,8500,8501,8502,8503,8504,8505,8506,8507,8508,8509,...,8590,8591,8592,8593,8594,8595,8596,8597,8598,8599
86,8600,8601,8602,8603,8604,8605,8606,8607,8608,8609,...,8690,8691,8692,8693,8694,8695,8696,8697,8698,8699
87,8700,8701,8702,8703,8704,8705,8706,8707,8708,8709,...,8790,8791,8792,8793,8794,8795,8796,8797,8798,8799
88,8800,8801,8802,8803,8804,8805,8806,8807,8808,8809,...,8890,8891,8892,8893,8894,8895,8896,8897,8898,8899
89,8900,8901,8902,8903,8904,8905,8906,8907,8908,8909,...,8990,8991,8992,8993,8994,8995,8996,8997,8998,8999
90,9000,9001,9002,9003,9004,9005,9006,9007,9008,9009,...,9090,9091,9092,9093,9094,9095,9096,9097,9098,9099
91,9100,9101,9102,9103,9104,9105,9106,9107,9108,9109,...,9190,9191,9192,9193,9194,9195,9196,9197,9198,9199
92,9200,9201,9202,9203,9204,9205,9206,9207,9208,9209,...,9290,9291,9292,9293,9294,9295,9296,9297,9298,9299
93,9300,9301,9302,9303,9304,9305,9306,9307,9308,9309,...,9390,9391,9392,9393,9394,9395,9396,9397,9398,9399
94,9400,9401,9402,9403,9404,9405,9406,9407,9408,9409,...,9490,9491,9492,9493,9494,9495,9496,9497,9498,9499


## Arithmetic <a name="arithmetic"></a>
When you are performing arithmetic operations between two objects, if any index/column pairs are not the same, the respective indexes/columns in the result will be the union of the index pair. Let's have a look

In [27]:
df1 = pd.DataFrame(np.arange(12).reshape((3,4)),
                  columns=list("abcd"))
df1

,a,b,c,d
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11


In [28]:
df2 = pd.DataFrame(np.arange(12).reshape((3,4)),
                  columns=list("cdef"))
df2

,c,d,e,f
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11


In [29]:
# adding the two
df1+df2

,a,b,c,d,e,f
0,NaN,NaN,2,4,NaN,NaN
1,NaN,NaN,10,12,NaN,NaN
2,NaN,NaN,18,20,NaN,NaN


Notice how where we don't have matching values from `df1` and `df2` the output of the addition operation is `NaN` since there are no two numbers to add.

Well, we can "fix" that by filling in the `NaN` values. This effectively tells pandas where there are no two values to add, assume that the missing value is just zero.

In [30]:
df1.add(df2, fill_value=0)

,a,b,c,d,e,f
0,0.0,1.0,2,4,2.0,3.0
1,4.0,5.0,10,12,6.0,7.0
2,8.0,9.0,18,20,10.0,11.0


Here's a list of all arithmetic operations within panda. They are similar to NumPy!

| Operator | Method | Description |
| -- | -- | -- |
| + | add | Addition |
| - | sub | Subtraction |
| / | div | Division |
| // | floordiv | Floor division |
| * | mul | Multiplication |
| ** | pow | Exponentiation |


### Exercise 2
Create a (3,3) DataFrame and square all elements in it.

In [31]:
pd.DataFrame(np.arange(9).reshape(3,3)) ** 2

,0,1,2
0,0,1,4
1,9,16,25
2,36,49,64


## Summarizing and computing descriptive stats <a name="sums"></a>
`pandas` is equipped with common mathematical and statistical methods. Most of which fall into the category of reductions or summary statistics. These are methods that extract a single value from a list of values. For example, you can extract the sum of a `Series` object like this:

In [32]:
df = pd.DataFrame(np.arange(20).reshape(5,4),
                 columns=["a", "b", "c", "d"])
df

,a,b,c,d
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


In [33]:
df.sum()

a    40
b    45
c    50
d    55
dtype: int64

Notice how that created the sum of each column?

Well you can actually make that the other way around by adding an extra option to `sum()`

In [34]:
df.sum(axis="columns")

0     6
1    22
2    38
3    54
4    70
dtype: int64

A similar method also exists for obtaining the mean of data:

In [35]:
df.mean()

a     8.0
b     9.0
c    10.0
d    11.0
dtype: float64

Finally, the mother of the methods we discussed here is `describe()` 

In [36]:
df.describe()

,a,b,c,d
count,5.000000,5.000000,5.000000,5.000000
mean,8.000000,9.000000,10.000000,11.000000
std,6.324555,6.324555,6.324555,6.324555
min,0.000000,1.000000,2.000000,3.000000
25%,4.000000,5.000000,6.000000,7.000000
50%,8.000000,9.000000,10.000000,11.000000
75%,12.000000,13.000000,14.000000,15.000000
max,16.000000,17.000000,18.000000,19.000000


Here are some of the summary methods:

| Method | Description |
| -- | -- |
| count          | Return number of non-NA values |
| describe       | Set of summary statistics |
| min, max       | Minimum, maximum values |
| argmin, argmax | Index locations at which the minimum or maximum value is obtained | 
| sum            | Sum of values |
| mean           | Mean of values |
| median         | Arithmetic median of values |
| std            | Sample standard deviation of values
| value_counts() | Counts the number of occurrences of each unique element in a column |

### Exercise 3

A random DataFrame is created below. Find the median value of each column.

In [37]:
df = pd.DataFrame(np.random.uniform(0, 10, (100, 100)))
df.median(axis=0)

0     4.207091
1     4.901982
2     5.499645
3     5.284451
4     5.178379
5     5.602175
6     5.032276
7     3.890377
8     4.837505
9     5.812539
10    5.305762
11    5.263855
12    4.537093
13    4.697431
14    4.678316
15    4.750120
16    5.184488
17    5.231531
18    4.230153
19    5.570728
20    6.171645
21    4.469543
22    4.670613
23    4.818360
24    5.465085
25    4.364515
26    4.576279
27    4.646400
28    4.756576
29    4.486468
        ...   
70    5.271716
71    5.523509
72    5.272633
73    5.385989
74    4.183016
75    4.906069
76    5.152698
77    4.794943
78    3.909044
79    5.639653
80    4.466024
81    4.995169
82    4.993824
83    5.292183
84    5.433545
85    4.819165
86    4.953971
87    4.275257
88    6.007742
89    5.700464
90    4.542109
91    5.943234
92    5.520957
93    4.745144
94    4.692347
95    4.591827
96    4.474495
97    5.033800
98    4.849259
99    5.215888
Length: 100, dtype: float64

# Data Loading and Storing <a name="loading"></a>
Accessing data is a necessary first step for data science.

A very common format is the `.csv`. This is an easy to read file format which is usually visualised like a spreadsheet. The data itself is usually separated with a `,` which is called the **delimiter**.

Here is an example of a `.csv` file:
```
name,sex
Joseph,M	
Andrew,M	
Joshua,M	
Abigail,F	
Benjamin,M	
Anthony,M	
James,M	
Emily,F	
Elijah,M	
Matthew,M	
Daniel,M	
Aiden,M	
Alexander,M	
Ava,F	
Jayden,M	
Michael,M	
Liam,M	
William,M	
Olivia,F	
Noah,M	
```

It details some names and sexes of newborn babies. The first line is called the header, and you can imagine that it is the name of the columns of a spreadsheet.

Let's now see how we can load this data and analyse it. The file is located in the folder `data` and is called `yob2012.txt`. We can read it like this:

In [38]:
babes = pd.read_csv("data/yob2012.csv")
babes

,name,sex
0,Sophia,F
1,Emma,F
2,Isabella,F
3,Olivia,F
4,Ava,F
5,Emily,F
6,Abigail,F
7,Mia,F
8,Madison,F
9,Elizabeth,F


Easy right?

### Exercise 4
Load the file `data/homes.csv` and find the mean selling price of these houses.

In [39]:
homes = pd.read_csv("data/homes.csv")
homes

,Sell,List,Living,Rooms,Beds,Baths,Age,Acres,Taxes
0,142,160,28,10,5,3,60,0.28,3167
1,175,180,18,8,4,1,12,0.43,4033
2,143,145,21,7,4,2,10,1.20,3529
3,129,132,13,6,3,1,41,0.33,1471
4,138,140,17,7,3,1,22,0.46,3204
5,232,240,25,8,4,3,5,2.05,3613
6,135,140,18,7,4,3,9,0.57,3028
7,150,160,20,8,4,3,18,4.00,3131
8,293,305,26,8,4,3,6,0.46,7088
9,207,225,22,8,4,2,16,2.22,5158


In [40]:
homes["Sell"].mean()

169.0

The `read_csv` function has a lot of optional arguments (more than 50). It's impossible to memorise all of them - it's usually best just to look up the particular functionality when you need it. 

You can search `pandas read_csv` online and find all of the documentation.

There are also many other functions that can read textual data. Here are some of them:

| Function | Description
| -- | -- |
| read_csv       | Load delimited data from a file, URL, or file-like object. The default delimiter is a comma `,` |
| read_table     | Load delimited data from a file, URL, or file-like object. The default delimiter is tab `\t` |
| read_clipboard | Reads the last object you have copied (Ctrl-C) |
| read_excel     | Read tabular data from Excel XLS or XLSX file |
| read_hdf       | Read HDF5 file written by pandas |
| read_html      | Read all tables found in the given HTML document |
| read_json      | Read data from a JSON string representation |
| read_sql       | Read the results of a SQL query |

*Note: there are also other loading functions which are not touched upon here*

# Data Cleaning <a name="cleaning"></a>
While doing data analysis and modeling, a significant amount of time is spent on data preparation: loading, cleaning, transforming and rearranging. Such tasks are often reported to take **up to 80%** or more of a data analyst's time. Often the way the data is stored in files isn't in the correct format and needs to be modified. Researchers usually do this on an ad-hoc basis using programming languages like Python.

In this chapter, we will discuss tools for handling missing data, duplicate data, string manipulation, and some other analytical data transformations.

## Handling missing data <a name="missing"></a>
Missing data occurs commonly in many data analysis applications. One of the goals of pandas is to make working with missing data as painless as possible.

In pandas, missing numeric data is represented by `NaN` (Not a Number) and can easily be handled:

In [41]:
string_data = pd.Series(['orange', 'tomato', np.nan, 'avocado'])
string_data

0     orange
1     tomato
2        NaN
3    avocado
dtype: object

In [42]:
string_data.isnull()

0    False
1    False
2     True
3    False
dtype: bool

The pandas `NaN` is functionally equlevant to the standard Python type `NoneType` which can be defined with `x = None`.

Here are some other methods which you can find useful:
    
| Method | Description |
| -- | -- |
| dropna | Filter axis labels based on whether the values of each label have missing data|
| fillna | Fill in missing data with some value |
| isnull | Return boolean values indicating which values are missing |
| notnull | Negation of isnull |

Just like `.drop()`, these methods all return a new object, leaving the original unchanged (this behaviour can be overridden using the argument `inplace=True`).

### Exercise 5
Remove the missing data below using the appropriate method

In [43]:
data = pd.Series([1, None, 3, 4, None, 6])
data

0    1.0
1    NaN
2    3.0
3    4.0
4    NaN
5    6.0
dtype: float64

In [44]:
data.dropna()

0    1.0
2    3.0
3    4.0
5    6.0
dtype: float64

`dropna()` by default removes any row/column that has a missing value. What if we want to remove only rows in which all of the data is missing though?

In [45]:
data = pd.DataFrame([[1., 6.5, 3.], [1., None, None],
                    [None, None, None], [None, 6.5, 3.]])
data

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [46]:
data.dropna()

,0,1,2
0,1.0,6.5,3.0


In [47]:
data.dropna(how="all")

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
3,NaN,6.5,3.0


### Exercise 6
That's fine if we want to remove missing data, what if we want to fill in missing data? Do you know of a way? Try to fill in all of the missing values from the data below with **0s**

In [48]:
data = pd.DataFrame([[1., 6.5, 3.], [2., None, None],
                    [None, None, None], [None, 1.5, 9.]])
data

,0,1,2
0,1.0,6.5,3.0
1,2.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,1.5,9.0


In [49]:
data.fillna(0)

,0,1,2
0,1.0,6.5,3.0
1,2.0,0.0,0.0
2,0.0,0.0,0.0
3,0.0,1.5,9.0


## Data Transformation  <a name="transformation"></a>
### Removing duplicates
Duplicate data can be a serious issue, luckily pandas offers a simple way to remove duplicates

In [50]:
data = pd.DataFrame([1, 2, 3, 4, 3, 2, 1])
data

,0
0,1
1,2
2,3
3,4
4,3
5,2
6,1


In [51]:
data.drop_duplicates()

,0
0,1
1,2
2,3
3,4


You can also select which rows to keep

In [52]:
data.drop_duplicates(keep="last")

,0
3,4
4,3
5,2
6,1


### Replacing data
You've already seen how you can fill in missing data with `fillna()`. That is actually a special case of more general value replacement. That is done via the `replace()` method.

Let's consider an example where the dataset given to us had `-999` as sentinel values for missing data instead of `NaN`.

In [53]:
data = pd.DataFrame([1., -999., 2., -999., 3., 4., -999, -999, 7.])
data

,0
0,1.0
1,-999.0
2,2.0
3,-999.0
4,3.0
5,4.0
6,-999.0
7,-999.0
8,7.0


In [54]:
data.replace(-999, np.nan)

,0
0,1.0
1,NaN
2,2.0
3,NaN
4,3.0
5,4.0
6,NaN
7,NaN
8,7.0


### Detecting and Filtering Outliers
Filtering or transforming outliers is largely a matter of applying array operations. Consider a DataFrame with some normally distributed data:

In [55]:
data = pd.DataFrame(np.random.randn(1000, 4))
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.001302,-0.012344,0.031341,0.010315
std,0.966097,1.000280,0.963462,1.029088
min,-3.768112,-3.227195,-2.806889,-3.364708
25%,-0.633164,-0.716059,-0.652919,-0.685110
50%,-0.042795,-0.060326,0.018802,0.018280
75%,0.640987,0.700712,0.706296,0.672465
max,3.352564,3.274630,3.534321,2.862938


Suppose you now want to lower all values exceeding 2 from one of the columns.

In [56]:
col = data[2]
col[col > 2]

56     3.534321
123    2.962325
270    2.039024
273    2.121346
368    2.098977
392    2.262474
426    2.583615
473    2.304300
487    2.237144
495    3.212527
577    2.147851
592    2.340029
642    2.171905
686    2.300286
724    2.472831
745    2.281898
761    2.129638
822    2.634474
862    2.086710
890    2.205046
936    2.201457
960    2.091273
979    2.098960
Name: 2, dtype: float64

We could set a ceiling for these values at 2.

In [57]:
data[data > 2] = 2
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.006558,-0.023270,0.022822,0.003351
std,0.947089,0.973852,0.942435,1.013547
min,-3.768112,-3.227195,-2.806889,-3.364708
25%,-0.633164,-0.716059,-0.652919,-0.685110
50%,-0.042795,-0.060326,0.018802,0.018280
75%,0.640987,0.700712,0.706296,0.672465
max,2.000000,2.000000,2.000000,2.000000


What we just did is called **boolean indexing**.

### Exercise 7
Let's load again our file with home prices and filter out homes based on our preference:
1. Load up the file `data/homes.csv`
2. The data contains some duplicates. Filter them out.
3. Let's say that the most we can spend on a house is £150. Keep only houses that have a **sell**ing price less than £150 and remove the rest
4. Select only houses that have 4 or more bedrooms
5. Select only houses that have 3 or more baths

You should end up with only 2 houses

In [58]:
homes = homes.drop_duplicates()
homes

,Sell,List,Living,Rooms,Beds,Baths,Age,Acres,Taxes
0,142,160,28,10,5,3,60,0.28,3167
1,175,180,18,8,4,1,12,0.43,4033
2,143,145,21,7,4,2,10,1.20,3529
3,129,132,13,6,3,1,41,0.33,1471
4,138,140,17,7,3,1,22,0.46,3204
5,232,240,25,8,4,3,5,2.05,3613
6,135,140,18,7,4,3,9,0.57,3028
7,150,160,20,8,4,3,18,4.00,3131
8,293,305,26,8,4,3,6,0.46,7088
9,207,225,22,8,4,2,16,2.22,5158


In [59]:
homes_in_budget = homes[homes["Sell"] < 150]
homes_in_budget

,Sell,List,Living,Rooms,Beds,Baths,Age,Acres,Taxes
0,142,160,28,10,5,3,60,0.28,3167
2,143,145,21,7,4,2,10,1.20,3529
3,129,132,13,6,3,1,41,0.33,1471
4,138,140,17,7,3,1,22,0.46,3204
6,135,140,18,7,4,3,9,0.57,3028
11,89,90,10,5,3,1,43,0.30,2054
13,87,90,16,7,3,1,50,0.65,1445
15,106,116,20,8,4,1,13,0.22,2818
19,136,140,19,7,3,1,22,0.63,3607
20,148,160,17,7,3,2,13,0.36,3648


In [60]:
homes_beds = homes_in_budget[homes_in_budget["Beds"] >= 4]
homes_beds

,Sell,List,Living,Rooms,Beds,Baths,Age,Acres,Taxes
0,142,160,28,10,5,3,60,0.28,3167
2,143,145,21,7,4,2,10,1.20,3529
6,135,140,18,7,4,3,9,0.57,3028
15,106,116,20,8,4,1,13,0.22,2818
28,110,115,16,8,4,1,26,0.29,3103
29,135,145,18,7,4,1,35,0.43,3363
40,127,130,20,8,4,1,65,0.40,3334
43,128,135,17,9,4,1,25,0.52,3374
44,110,120,15,8,4,2,11,0.59,3119
45,123,130,18,8,4,2,43,0.39,3268


In [61]:
homes_baths_beds = homes_beds[homes_beds["Baths"] >= 3]
homes_baths_beds

,Sell,List,Living,Rooms,Beds,Baths,Age,Acres,Taxes
0,142,160,28,10,5,3,60,0.28,3167
6,135,140,18,7,4,3,9,0.57,3028
